In [1]:
import warnings
import regex as re
from pathlib import Path
import spacy
from spacy import displacy
import pandas as pd
import numpy as np
import coreferee
from sentence_transformers import SentenceTransformer, util
from nltk.translate.bleu_score import SmoothingFunction


def resolve_coreference(text):
    doc = nlp(text)
    doc_list = list(doc)
    # doc._.coref_chains.print()
    resolving_indecies = []
    for _,item in enumerate(doc._.coref_chains):
        resolving_indecies.extend(item)
        
    for word in resolving_indecies:
        new_word = ""
        for index in word:
            if doc[index]._.coref_chains.resolve(doc[index]) is not None:
                temp = []
                for item in doc._.coref_chains.resolve(doc[index]):
                    temp.append(str(item))
                new_word = ", ".join(temp)
            
                doc_list[index] = new_word

    final_doc = []
    for item in doc_list:
        final_doc.append(str(item))
    return " ".join(final_doc)


def extract_subjects(sentence):
    subjects = []
    for token in sentence:
        if token.dep_ in ("nsubj","csubj", "nsubjpass","relcl"):
            if token.dep_ == "relcl":
                # The head of the token with the "relcl" dependency is the noun that the relative clause is modifying
                subject = str(token.head)
                verb = token
                subjects.append((subject, verb))
            elif token.dep_ == "nsubjpass":
                verb = token.head
                for child in verb.children:
                    if child.dep_ == "agent":
                        subject = [str(t) for t in list(child.children)[0].subtree]
                        subject = " ".join(subject)
                        break
                    else:
                        subject = "Unknown"
                subjects.append((subject, verb))
            else:                       
                subtree_tokens = [str(t) for t in token.subtree]
                verb = token.head
                subjects.append((" ".join(subtree_tokens), verb))
    return subjects

def extract_objects(sentence):
    objects = []
    for token in sentence:
        if token.dep_ in ("dobj", "dative", "attr", "oprd", "acomp","ccomp", "xcomp", "nsubjpass"):
            subtree_tokens = [str(t) for t in token.subtree]
            verb = token.head
            objects.append((" ".join(subtree_tokens), verb))
    return objects

def extract_state(sentence):
    states = []
    for token in sentence:
        if token.pos_ =="VERB" or token.pos_ == "AUX":
            for child in token.children:
                if child.dep_ == "prep":
                    subtree_tokens = [str(t) for t in child.subtree if t != child]
                    states.append(((" ".join(subtree_tokens), token)))
    return states

def extract_time(sentence):
    times = {}
    for token in sentence:
        if token.pos_ == "VERB" or token.pos_ == "AUX":
            for child in token.subtree:
                if child.ent_type_ == "DATE" or child.ent_type_ == "TIME":
                    times[child.text] = token
    return list(times.items())

def extract_location(sentence):
    locations = {}
    for token in sentence:
        if token.pos_ == "VERB" or token.pos_ == "AUX":
            for child in token.subtree:
                if child.ent_type_ in ("GPE", "LOC", "FAC"):
                    locations[child.text] = token
    return list(locations.items())
                    
    

def extract_facts(sentence):
    sentence = nlp(sentence)
    states = extract_state(sentence)
    subjects = extract_subjects(sentence)
    objects = extract_objects(sentence)
    times = extract_time(sentence)
    locations = extract_location(sentence)
    # print(subjects, objects)
    
    facts = pd.DataFrame(columns=["Subject", "Relation", "Objects", "States", "Times", "Locations"])
    
    for subject in subjects:
        verb = subject[1].lemma_
        currentSubject = subject[0]
        if verb in facts["Relation"].values:
            facts.loc[facts["Relation"] == verb, "Subject"] = currentSubject
        else:
            new_row = pd.DataFrame([{"Subject": currentSubject, "Relation": verb, "Objects": [], "States": [], "Times": [], "Locations": []}])
            facts = pd.concat([facts, new_row], ignore_index=True)
    for obj in objects:
        verb = obj[1].lemma_
        currentObj = obj[0]
        if verb in facts["Relation"].values:
            oldObjects = list(facts.loc[facts["Relation"] == verb, "Objects"].values[0])
            oldObjects.append(currentObj)
            facts.loc[facts["Relation"] == verb, "Objects"] = [oldObjects] 
            

    for state in states:
        verb = state[1].lemma_
        currentState = state[0]
        if verb in facts["Relation"].values:
            oldStates = list(facts.loc[facts["Relation"] == verb, "States"].values[0])
            oldStates.append(currentState)
            facts.loc[facts["Relation"] == verb, "States"] = [oldStates]
            
    for time in times:
        verb = time[1].lemma_
        currentTime = time[0]
        if verb in facts["Relation"].values:
            oldTimes = list(facts.loc[facts["Relation"] == verb, "Times"].values[0])
            oldTimes.append(currentTime)
            facts.loc[facts["Relation"] == verb, "Times"] = [oldTimes]
            
    for location in locations:
        verb = location[1].lemma_
        currentLocation = location[0]
        if verb in facts["Relation"].values:
            oldLocations = list(facts.loc[facts["Relation"] == verb, "Locations"].values[0])
            oldLocations.append(currentLocation)
            facts.loc[facts["Relation"] == verb, "Locations"] = [oldLocations]
            
    return facts
        
def preprocess_context(doc):
    text = doc.strip()
    text.replace(".", ",")
    resolved_text = resolve_coreference(text)
    resolved_text = resolved_text.strip()
    resolved_text = resolved_text.replace("  ", " ").replace(" ,", ",").replace(" .", ".").replace("\n", "")
    return resolved_text

def join_sentences_facts(sentences):
    all_facts = pd.DataFrame(columns=["Subject", "Relation", "Objects", "States", "Times", "Locations"])
    for sentence in sentences:
        facts = extract_facts(sentence)
        all_facts = pd.concat([all_facts, facts])
    all_facts = all_facts.groupby(["Subject", "Relation"], as_index=False).agg({
        "Objects": lambda x: [item for sublist in x for item in sublist],
        "States": lambda x: [item for sublist in x for item in sublist],
        "Times": lambda x: [item for sublist in x for item in sublist],
        "Locations": lambda x: [item for sublist in x for item in sublist]
    })
    return all_facts

def change_subject_relation(factsDF):
    for index, row in factsDF.iterrows():
        factsDF.loc[index, "Subject"] = [row['Subject']]
        factsDF.loc[index, "Relation"] = [row['Relation']]
    return factsDF

def similarity(factRow, questionRow, column):
    if len(factRow[column]) == 0 or len(questionRow[column]) == 0:
        return 0
    columnString = " ".join(factRow[column])
    questionString = " ".join(questionRow[column])
    embeddingFact = model.encode(columnString)
    embeddingQuestion = model.encode(questionString)
    return util.cos_sim(embeddingFact, embeddingQuestion)

        
def cost_function(factsDf, questionFact, excludeColumns=[]):
    score = 0
    maxFactIdx = 0
    columnNames = ["Subject","Relation", "Objects", "States", "Times", "Locations"]
    for column in excludeColumns:
        columnNames.remove(column)
    for factIdx, factRow in factsDf.iterrows():
        currScore = 0
        for _, questionRow in questionFact.iterrows():
            if len(factRow[excludeColumns[0]]) == 0:
                continue
            for column in columnNames:
                currScore += similarity(factRow, questionRow, column)
        if currScore > score:
            score = currScore
            maxFactIdx = factIdx
    return maxFactIdx, score

def process_question_context(question, doc):
    splitted_question = question.split(" ")
    question_type = splitted_question[0].lower()
    question_nlp = nlp(question)
    if question_nlp[0].ent_type_ == "DATE":
        question_type = "when"
    resolved_doc = preprocess_context(doc)
    cleaned_doc = nlp(resolved_doc)
    sentences = [one_sentence.text.strip() for one_sentence in cleaned_doc.sents]
    
    questionDF = extract_facts(question)
    factsDF = join_sentences_facts(sentences)
    
    newFactsDF = change_subject_relation(factsDF)
    newQuestionDF = change_subject_relation(questionDF)
    
    return newFactsDF, newQuestionDF, question_type

def get_answer(factsDF, questionDF, question_type):
    correctIdx, _ = cost_function(factsDF, questionDF, excludeColumns=[excludesPerQuestionType[question_type]])
    answer = factsDF.loc[correctIdx, excludesPerQuestionType[question_type]]
    if answer == []:
        answer = factsDF.loc[correctIdx, "States"]    
    return " ".join(answer)
    

# if __name__ == "__main__":
nlp = spacy.load('en_core_web_md')
nlp.add_pipe("merge_entities")
nlp.add_pipe("merge_noun_chunks")
nlp.add_pipe('coreferee')
model = SentenceTransformer("all-MiniLM-L6-v2")

excludesPerQuestionType = {
    "when": "Times",
    "where": "Locations",
    "who": "Subject",
    "what": "Objects",
    "how": "States"
}   
    
# doc = """
# Lionel Andrés "Leo" Messi was born in 24 June 1987 is an Argentine professional footballer plays as a forward for and captains both Major League Soccer club Inter Miami and the Argentina national team.
# He played in Barcelona in 2010.
# Widely regarded as one of the greatest players of all time, Messi has won a record eight Ballon d'Or awards, a record six European Golden Shoes, and was named the world's best player for a record eight times by FIFA.
# Until 2021, he had spent his entire professional career with Barcelona, where he won a club-record 34 trophies, including ten La Liga titles, seven Copa del Rey titles, and the UEFA Champions League four times.
# With his country, he won the 2021 Copa América and the 2022 FIFA World Cup. A prolific goalscorer and creative playmaker, Messi holds the records for most goals, hat-tricks, and assists in La Liga. He has the most international goals by a South American male. Messi has scored over 800 senior career goals for club and country, and the most goals for a single club.
# """
# question = "how did messi play?"
# factsDF, questionDF, question_type = process_question_context(question, doc)
# answer = get_answer(factsDF, questionDF, question_type)

# print("========================================================")
# print("Question: ", question)
# print("Answer: ", answer)


c:\Users\zisak\anaconda3\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\zisak\anaconda3\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\zisak\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\zisak\anaconda3\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch

In [2]:
import random
from datasets import load_dataset

dataset = load_dataset("rajpurkar/squad")
# train = dataset['train']
validation = dataset['validation']

random.seed(42)
import getopt
import sys

import pandas as pd
import json 
import regex as re
from tqdm import tqdm
import nltk

In [3]:

def QuestionStartsWith_Accuracy(dataset, startsWith):

    correct = 0
    EM = 0
    BLEU = 0
    total = 0
    errors = []
    corrects = []
    empties = []
    kolo = 0
    for item in tqdm(dataset):
        random_number = random.randint(0, len(dataset))
        kolo += 1
        included_question = False
        try:
            context = item['context']
            context = re.sub(' +', ' ', context)
            
            question = item['question']
            tempQuestion = question.lower()
            question = re.sub(' +', ' ', question)
            
            answer = item['answers']['text'][0]
            title = item['title']
    
            # check if question starts with startsWith
            for start in startsWith:
                if tempQuestion.startswith(start):
                    included_question = True
                    break
            
            if included_question:
    
                total += 1
                if total == 500:
                    break
                    
                factsDF, questionDF, question_type = process_question_context(question, context)
                outputAnswer = get_answer(factsDF, questionDF, question_type)
                if outputAnswer == "":
                    empties.append(kolo)
                    outputAnswer = "No_Answer_Found"
                
                # print("Question: " , question)
                # print("Answer: ", answer, "-------" , "Our Answer: ", outputAnswer)
                if outputAnswer in answer or answer in outputAnswer:
                    correct += 1
                    corrects.append(kolo)
                else:
                    print("Question: " , question)
                    print("Answer: ", answer, "-------" , "Our Answer: ", outputAnswer)
                if outputAnswer == answer:
                    EM += 1

                n = min(len(outputAnswer.split()), 4)
                if n == 0:
                    BLEUscore = 0
                else:
                    weights = [1.0/n]*n
                    smoothie = SmoothingFunction().method4
                    BLEUscore = nltk.translate.bleu_score.sentence_bleu([answer], outputAnswer, weights=weights, smoothing_function=smoothie)
                    BLEU += BLEUscore
                    

            if kolo % 1000 == 0:
                print(f"Correct: {correct}, out of {total}: {100*correct/total}%")
                print(f"EM: {EM}, out of {total}: {100*EM/total}%")
                print(f"BLEU: {BLEU}, out of {total}: {100*BLEU/total}%")
        
        except Exception as e:
            print("title: ", title)
            print("Error in question number: ", total)
            print("Question: ", question)
            print("Answer: ", answer)
            print("Error: ", e)
            errors.append(total)
            # total -= 1
            print("\n\n")
    
    if total != 0:
        print(f"Correct: {correct}, out of {total}: {100*correct/total}%")
        print(f"EM: {EM}, out of {total}: {100*EM/total}%")
        print(f"BLEU: {BLEU}, out of {total}: {100*BLEU/total}%")
    else:
        print("No Questions found with the given starting word")
    print("Errors: ", errors)
    print("Empties: ", len(empties), empties)
    return correct, total, errors, corrects


if __name__ == "__main__":

    startsWith = ["what "]
    x = QuestionStartsWith_Accuracy(validation, startsWith)



  0%|          | 5/10570 [00:00<21:29,  8.19it/s]

Question:  What color was used to emphasize the 50th anniversary of the Super Bowl?
Answer:  gold ------- Our Answer:  The game


  0%|          | 6/10570 [00:01<37:39,  4.68it/s]

Question:  What was the theme of Super Bowl 50?
Answer:  "golden anniversary" ------- Our Answer:  an American football game to determine the champion of the National Football League ( NFL ) for the 2015 season


  0%|          | 8/10570 [00:02<56:49,  3.10it/s]

...

Question:  What was Tesla's position at the Budapest Telephone Exchange?
Answer:  chief electrician ------- Our Answer:  functional


 12%|█▏        | 1309/10570 [04:40<45:33,  3.39it/s]

Question:  What was Edison's reply as to what Tesla didn't understand when Tesla asked for payment?
Answer:  American humor. ------- Our Answer:  that Tesla could redesign Edison's inefficient motor and generators , making an improvement in both service and economy


 12%|█▏        | 1312/10570 [04:41<33:05,  4.66it/s]

Correct: 109, out of 500: 21.8%
EM: 18, out of 500: 3.6%
BLEU: 67.56995938346863, out of 500: 13.513991876693726%
Errors:  []
Empties:  1 [898]


In [4]:
# [174, 315, 326, 349, 472, 488, 493, 498, 530, 667]
# when: 48%